<a href="https://colab.research.google.com/github/ISaySalmonYouSayYes/BA_thesis/blob/main/BERT_fakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Before you start**



In [ ]:
try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

import os

num_cpus = os.cpu_count()
print(f"Number of available CPU cores: {num_cpus}")


Note: using Google CoLab
Number of available CPU cores: 12


In [ ]:
!pip install transformers datasets scikit-learn torch

# **1. Load training data**  
- **train_DataSet:** dataset["train"]
- **test_DataSet:** dataset["test"]
- **validation_DataSet:** dataset["validation"]

In [ ]:
from datasets import load_dataset

dataset_orig = load_dataset("ucsbnlp/liar")
print(dataset_orig)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

liar.py:   0%|          | 0.00/6.41k [00:00<?, ?B/s]

The repository for ucsbnlp/liar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ucsbnlp/liar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/10269 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1283 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1284 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 10269
    })
    test: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1283
    })
    validation: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1284
    })
})


In [ ]:
for i in range(100):
    print(dataset_orig['train'][i]) #label: true, false, half-true, pants-fire, barely-true, mostly-true (False:0=>5:True, I guess)

{'id': '2635.json', 'label': 0, 'statement': 'Says the Annies List political group supports third-trimester abortions on demand.', 'subject': 'abortion', 'speaker': 'dwayne-bohac', 'job_title': 'State representative', 'state_info': 'Texas', 'party_affiliation': 'republican', 'barely_true_counts': 0.0, 'false_counts': 1.0, 'half_true_counts': 0.0, 'mostly_true_counts': 0.0, 'pants_on_fire_counts': 0.0, 'context': 'a mailer'}
{'id': '10540.json', 'label': 1, 'statement': 'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.', 'subject': 'energy,history,job-accomplishments', 'speaker': 'scott-surovell', 'job_title': 'State delegate', 'state_info': 'Virginia', 'party_affiliation': 'democrat', 'barely_true_counts': 0.0, 'false_counts': 0.0, 'half_true_counts': 1.0, 'mostly_true_counts': 1.0, 'pants_on_fire_counts': 0.0, 'context': 'a floor speech.'}
{'id': '324.json', 'label': 2, 'statement': 'Hillary C

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset


# Convert each split into Pandas DataFrames and keep relevant columns
df_train = pd.DataFrame(dataset_orig["train"])[['statement', 'label']]
df_test = pd.DataFrame(dataset_orig["test"])[['statement', 'label']]
df_validation = pd.DataFrame(dataset_orig["validation"])[['statement', 'label']]

# Map the labels to binary: Fake (0) and Real (1)
# def assign_fake_true_labels(label):
#     return float(0) if label < 2 else float(1)

# df_train['label'] = df_train['label'].apply(assign_fake_true_labels)
# df_test['label'] = df_test['label'].apply(assign_fake_true_labels)
# df_validation['label'] = df_validation['label'].apply(assign_fake_true_labels)

# **2. Preprocess the data**

In [ ]:
from transformers import BertTokenizer

# Convert Pandas DataFrames back to Hugging Face Datasets
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)
val_dataset = Dataset.from_pandas(df_validation)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define the tokenization function
def tokenize_function(sample):
    return tokenizer(sample['statement'], padding="max_length", truncation=True, max_length=256)

# Apply tokenization to datasets
train_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=12)
test_dataset = test_dataset.map(tokenize_function, batched=True, num_proc=12)
val_dataset = val_dataset.map(tokenize_function, batched=True, num_proc=12)

# Set the format to PyTorch tensors for Trainer compatibility
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map (num_proc=12):   0%|          | 0/10269 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/1283 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/1284 [00:00<?, ? examples/s]

# **3. BERT core method**

In [ ]:
from transformers import BertForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from transformers import Trainer, TrainingArguments

# Load BERT model with a classification head
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

# Define Evaluation Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to predicted class labels using argmax
    preds = torch.argmax(torch.tensor(logits), axis=-1).numpy()
    labels = np.array(labels)

    # Compute multi-class metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro", zero_division=0  # Use 'macro' for multi-class
    )
    acc = accuracy_score(labels, preds)

    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# 5. Set Up Training Arguments
training_args = TrainingArguments(
    output_dir="./results",          # Directory for saving model checkpoints
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save the model every epoch
    learning_rate=2e-5,              # Recommended learning rate for BERT
    per_device_train_batch_size=32,  # Batch size for training
    per_device_eval_batch_size=32,   # Batch size for evaluation
    num_train_epochs=4,              # Number of epochs
    weight_decay=0.01,               # L2 regularization
    logging_dir="./logs",            # Directory for logs
    logging_steps=10,                # Log every 10 steps
    load_best_model_at_end=True,     # Load the best model after training
    report_to="none",                 # Disable W&B logging
    fp16=True,
    )

# 6. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# **4. Fine-tuned the model**

In [ ]:
# 7. Train the Model
trainer.train()

# 8. Evaluate the Model
print("Evaluation on Validation Set:")
results_val = trainer.evaluate(val_dataset)
print(results_val)

print("Evaluation on Test Set:")
results_test = trainer.evaluate(test_dataset)
print(results_test)

# 9. Save the Model
model.save_pretrained("./fake_news_bert_model")
tokenizer.save_pretrained("./fake_news_bert_model")

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.710600,1.698477,0.259346,0.310605,0.230466,0.204107
2,1.622800,1.674387,0.265576,0.269231,0.249589,0.248470
3,1.466700,1.731893,0.271028,0.286949,0.267701,0.261526
4,1.355400,1.755432,0.274922,0.282450,0.269055,0.266736


Evaluation on Validation Set:


{'eval_loss': 1.6743870973587036, 'eval_accuracy': 0.26557632398753894, 'eval_precision': 0.269230685071784, 'eval_recall': 0.24958912412068154, 'eval_f1': 0.24847028445069866, 'eval_runtime': 2.326, 'eval_samples_per_second': 552.011, 'eval_steps_per_second': 17.627, 'epoch': 4.0}
Evaluation on Test Set:
{'eval_loss': 1.6474107503890991, 'eval_accuracy': 0.2946219797349961, 'eval_precision': 0.3123171934255134, 'eval_recall': 0.2803738599589593, 'eval_f1': 0.28093317230409376, 'eval_runtime': 2.3753, 'eval_samples_per_second': 540.137, 'eval_steps_per_second': 17.261, 'epoch': 4.0}


('./fake_news_bert_model/tokenizer_config.json',
 './fake_news_bert_model/special_tokens_map.json',
 './fake_news_bert_model/vocab.txt',
 './fake_news_bert_model/added_tokens.json')

# **5. Playground**

In [ ]:
# 10. Inference on New Text
def predict_new_text(text):
    # Check model device (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Ensure the model is on the correct device

    # Tokenize the input text and move to the model's device
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the correct device

    # Perform inference
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()

    # Convert prediction to label
    label = "Fake News" if prediction == 0 else "Real News"
    return prediction

# Example of inference
df_validation = pd.DataFrame(dataset_orig["validation"])[['statement', 'label']]
for text in df_validation['statement']:
  prediction = predict_new_text(text)
  print(f"Prediction for new text: {prediction}")

Prediction for new text: 2
Prediction for new text: 5
Prediction for new text: 0
Prediction for new text: 2
Prediction for new text: 0
Prediction for new text: 2
Prediction for new text: 1
Prediction for new text: 4
Prediction for new text: 0
Prediction for new text: 4
Prediction for new text: 4
Prediction for new text: 2
Prediction for new text: 2
Prediction for new text: 2
Prediction for new text: 0
Prediction for new text: 1
Prediction for new text: 2
Prediction for new text: 3
Prediction for new text: 1
Prediction for new text: 5
Prediction for new text: 2
Prediction for new text: 0
Prediction for new text: 3
Prediction for new text: 1
Prediction for new text: 1
Prediction for new text: 2
Prediction for new text: 2
Prediction for new text: 0
Prediction for new text: 2
Prediction for new text: 2
Prediction for new text: 0
Prediction for new text: 3
Prediction for new text: 2
Prediction for new text: 1
Prediction for new text: 0
Prediction for new text: 2
Prediction for new text: 1
P